In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import h5py
import glob
from torchvision import transforms
from PIL import Image
import os
import torchvision.models as models
from torch.nn.utils.rnn import pack_padded_sequence
import numpy as np
import torch.utils.data as data
import pandas as pd
import string
import difflib
import random
from collections import Counter
import nltk
import pickle
import math
import time
import matplotlib.pyplot as plt
import os.path
from os import path
!pip install bnltk
from bnltk.tokenize import Tokenizers
import re
torch.cuda.get_device_name()

'Tesla K80'

# Defining Directories 

In [3]:
caption_path =  '/content/drive/MyDrive/Thesis/code/Bangla MSR Video Description Corpus.xlsx'      # directory of the csv file
vocab_path = '/content/drive/MyDrive/Thesis/code/bangla_vocab.pkl'       # directory for saving vocabulary object (word to id dictionary)
threshold = 3                          # minimum frequency of a word for inclution in the dictionary


model_path = '/content/drive/MyDrive/Thesis/code/models/'
checkpoint_path = '/content/drive/MyDrive/Thesis/code/checkpoints/'
training_file_path = '/content/drive/MyDrive/Thesis/code/models/training.txt'
h5_file_path = '/content/drive/MyDrive/Thesis/code/features.h5'
resnext_file_path = '/content/drive/MyDrive/Thesis/code/resnext_101_features.h5'
embedding_path = '/content/drive/MyDrive/Thesis/code/bangla_w2v_embeddings.pkl'

shared_feature_file = '/content/drive/MyDrive/Thesis/code/features.h5'
shared_3dCNN_feature_file = '/content/drive/MyDrive/Thesis/code/resnext_101_features.h5'

# **Feature Extraction**

In [ ]:
!cp "/content/drive/My Drive/Thesis/resized equal number frames.zip" "/content/resized equal number frames.zip" 
!unzip "/content/resized equal number frames.zip"
!rm "/content/resized equal number frames.zip" 

Streaming output truncated to the last 5000 lines.
  inflating: resized equal number frames/0hyZ__3YhZc_380_384_8.jpg  
  inflating: resized equal number frames/0hyZ__3YhZc_410_417_3.jpg  
  inflating: resized equal number frames/0hyZ__3YhZc_575_580_16.jpg  
  inflating: resized equal number frames/0hyZ__3YhZc_632_637_9.jpg  
  inflating: resized equal number frames/0IDJG0q9j_k_1_24_1.jpg  
  inflating: resized equal number frames/0IDJG0q9j_k_1_24_7.jpg  
  inflating: resized equal number frames/0lh_UWF9ZP4_138_145_8.jpg  
  inflating: resized equal number frames/0lh_UWF9ZP4_157_160_17.jpg  
  inflating: resized equal number frames/0lh_UWF9ZP4_191_197_10.jpg  
  inflating: resized equal number frames/0lh_UWF9ZP4_199_207_23.jpg  
  inflating: resized equal number frames/0lh_UWF9ZP4_21_26_20.jpg  
  inflating: resized equal number frames/0lh_UWF9ZP4_21_26_3.jpg  
  inflating: resized equal number frames/0lh_UWF9ZP4_38_46_10.jpg  
  inflating: resized equal number frames/0lh_UWF9ZP4_62_69

In [ ]:
os.chdir(r'/content/resized equal number frames/')
all_frames = glob.glob("*.jpg")
dataset = set()

for item in all_frames:
  words = item.split('_')
  unique_name = words[0] 
  for i in range(1,len(words)-1):
    unique_name = unique_name + "_" + words[i] 
  dataset.add(unique_name)

print(len(dataset))

1970


In [ ]:
 transform = transforms.Compose([  
        transforms.ToTensor(), 
        transforms.Normalize((0.485, 0.456, 0.406), 
                             (0.229, 0.224, 0.225))])

In [ ]:
class EncoderCNN(nn.Module):
    def __init__(self):
        super(EncoderCNN, self).__init__()
        resnet = models.resnet152(pretrained=True)
        modules = list(resnet.children())[:-1]      # delete the last fc layer.
        self.resnet = nn.Sequential(*modules)
        
    def forward(self, images):
        """Extract feature vectors from input images."""
        with torch.no_grad():
            features = self.resnet(images)
        features = features.reshape(features.size(0), -1)
        return features

In [ ]:
encoder = EncoderCNN()
cnt = 1

done = []
with h5py.File(shared_feature_file, 'r') as hdf:
      done = list(hdf.keys())
     

for item in dataset:
    if item in done:
      continue
    images = []
    for i in range(1,33):
      image = Image.open(os.path.join('/content/resized equal number frames/',item+'_'+str(i)+'.jpg')).convert('RGB')
      image = transform(image)
      images.append(image)
    
    images = torch.stack(images,0)
    features = encoder(images)
    
    with h5py.File(shared_feature_file, 'a') as hdf:
      hdf.create_dataset(item, data = features)

    # print(cnt)
    # cnt += 1
    

# **Building Vocabulary**

In [4]:
class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
def build_vocab(csv, threshold):
    """Build a simple vocabulary wrapper."""
    #df = pd.read_csv('./msdv dataset/MSR Video Description Corpus.csv')
    df = pd.read_excel(csv)
    #df = df[df['Language'] == 'English']
    captions = df['Description']

    counter = Counter()
    for i, caption in enumerate(captions):
        caption = str(caption)
        tokens = nltk.tokenize.word_tokenize(caption.lower())
        tokens = [word for word in tokens if word.isalpha()]
        counter.update(tokens)

        if (i+1) % 1000 == 0:
            print("[{}/{}] Tokenized the captions.".format(i+1, len(captions)))

    
    # If the word frequency is less than 'threshold', then the word is discarded.
    words = [word for word, cnt in counter.items() if cnt >= threshold]

    # Create a vocab wrapper and add some special tokens.
    vocab = Vocabulary()
    vocab.add_word('<pad>')
    vocab.add_word('<start>')
    vocab.add_word('<end>')
    vocab.add_word('<unk>')

    # Add the words to the vocabulary.
    for i, word in enumerate(words):
        vocab.add_word(word)
    return vocab    
        
        
def main(caption_path,vocab_path,threshold):
    vocab = build_vocab(csv=caption_path, threshold=threshold)
    vocab_path = vocab_path
    with open(vocab_path, 'wb') as f:
        pickle.dump(vocab, f)
    print("Total vocabulary size: {}".format(len(vocab)))
    print("Saved the vocabulary wrapper to '{}'".format(vocab_path))

nltk.download('punkt')
main(caption_path, vocab_path,threshold)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[1000/85550] Tokenized the captions.
[2000/85550] Tokenized the captions.
[3000/85550] Tokenized the captions.
[4000/85550] Tokenized the captions.
[5000/85550] Tokenized the captions.
[6000/85550] Tokenized the captions.
[7000/85550] Tokenized the captions.
[8000/85550] Tokenized the captions.
[9000/85550] Tokenized the captions.
[10000/85550] Tokenized the captions.
[11000/85550] Tokenized the captions.
[12000/85550] Tokenized the captions.
[13000/85550] Tokenized the captions.
[14000/85550] Tokenized the captions.
[15000/85550] Tokenized the captions.
[16000/85550] Tokenized the captions.
[17000/85550] Tokenized the captions.
[18000/85550] Tokenized the captions.
[19000/85550] Tokenized the captions.
[20000/85550] Tokenized the captions.
[21000/85550] Tokenized the captions.
[22000/85550] Tokenized the captions.
[23000/85550] Tokenized the captions.
[24000/85550] Tokenized 

# Generate Train, Val and Test set

In [10]:
# Function for finding max overlapping subsequence between two strings
def get_overlap(s1, s2):
    s = difflib.SequenceMatcher(None, s1, s2)
    pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
    return s1[pos_a:pos_a+size]

# get all the video names with duration
with h5py.File(shared_feature_file, 'r') as hdf:
    ls = list(hdf.keys())
    #print('List of datasets in this file: \n', len(ls))

dataset = set()

# get unique vide IDs
for item in ls:
  words = item.split('_')
  unique_name = words[0] 
  for i in range(1,len(words)-2):
    unique_name = unique_name + "_" + words[i] 
  dataset.add(unique_name)



df2 = pd.read_excel(caption_path)
#df2 = df2[df2['Language'] == 'English']
ls3 = df2.VideoID.unique()


absent = [item for item in dataset if item not in ls3 ]
present = [item for item in dataset if item in ls3 ]

for item in absent:
  for item2 in ls:
    if len(get_overlap(item, item2)) > 4 :
      #print(item,item2)
      ls.remove(item2)


print(len(ls))
random.shuffle(ls)
cnt = 0
df = pd.read_excel(caption_path)
for item in ls[:1200]:
  splited_name = item.split('_')
  start = splited_name[-2]
  end = splited_name[-1]
  extention_len = len(start+end)+2
  only_vid_name = item[:-extention_len]
  #print(only_vid_name)

  #print('training data ',cnt) = item.split('_')
  start = splited_name[-2]
  end = splited_name[-1]
  extention_len = len(start+end)+2
  only_vid_name = item[:-extention_len]
  #print(only_vid_name)
  captions = df[df['VideoID'] == only_vid_name]
 # captions = captions[captions['Language'] == 'English']
  captions = captions[captions['Start'] == int(start)]
  captions = captions['Description']
  #print(len(captions))
  
  if len(captions) >0:
    with open(model_path+'training.txt', 'a') as file:  # Use file to refer to the file object
      for i in range(len(captions)):
        print(item, captions.iloc[i])
        if not isinstance(captions.iloc[i], str):
          continue
        s = captions.iloc[i]
        s = s.rstrip('\r')
        s = s.rstrip('\n')
        s = s.replace('\r', ' ')
        s = s.replace('\n', ' ')
        s = str(s.lower())
        file.write(item+'_'+str(i+1)+'\t'+s+'\n')
        cnt += 1

for item in ls[1200:1300]:
    with open(model_path+'validation.txt', 'a') as file:  # Use file to refer to the file object
      file.write(item+'\n')

for item in ls[1300:]:
    with open(model_path+'test.txt', 'a') as file:  # Use file to refer to the file object
      file.write(item+'\n')


print('training data ',cnt)

Streaming output truncated to the last 5000 lines.
pfQPRXcihkI_160_184 একজন লোক দেখিয়ে দিচ্ছেন কিভাবে আলু কাটা যায়
pfQPRXcihkI_160_184 একজন মহিলা আলু কাটছেন।
pfQPRXcihkI_160_184 একজন লোক ঘরে তৈরি ফ্রেঞ্চ ফ্রাই প্রস্তুত করছে।
pfQPRXcihkI_160_184 একজন লোক কিছু আলুর টুকরো টুকরো করছে।
pfQPRXcihkI_160_184 কেউ আলুর টুকরো টুকরো করে ফেলে।
pfQPRXcihkI_160_184 একজন মানুষ কিছু পটেটো কাটছে।
pfQPRXcihkI_160_184 একজন লোক কিছু পোটোটো কাটছে।
pfQPRXcihkI_160_184 একজন লোক কিছু আলুর টুকরো টুকরো করছে।
5K7Ru5s6YIw_8_11 এটার খাবারে একটি ক্যাঙ্গারু নিবলল।
5K7Ru5s6YIw_8_11 তৃণভূমিতে বসে একটি বাচ্চা ক্যাঙ্গারু তার সামনের পায়ে রাখা খাবার খাচ্ছে।
5K7Ru5s6YIw_8_11 একটি ক্যাঙ্গারু এক টুকরো রুটির মতো দেখতে খায়।
5K7Ru5s6YIw_8_11 একটি কাঙারু ঘাসে খাচ্ছে।
5K7Ru5s6YIw_8_11 একটি ক্যাঙ্গারু কিছু খাচ্ছে।
5K7Ru5s6YIw_8_11 একটা ক্যাঙ্গারু খাচ্ছে।
5K7Ru5s6YIw_8_11 একটা ক্যাঙ্গারু খাচ্ছে।
5K7Ru5s6YIw_8_11 একটা ক্যাঙ্গারু খাচ্ছে।
5K7Ru5s6YIw_8_11 একটা ক্যাঙ্গারু খাচ্ছে।
5K7Ru5s6YIw_8_11 একটা ক্যাঙ্গারু খাচ্ছে।
5K7Ru5s6YIw_

# Data Loader

In [11]:
class msvdDataset (data.Dataset):

  def __init__(self, root, vidFeature_path, txt, vocab):
    """Set the path for frame features, captions and vocabulary wrapper.
        
        Args:
            root: path for hdf5 file of features
            txt: bangla msvd annotation file path.
            vocab: vocabulary wrapper.
        """
    self.root = root
    self.vidFeature_path = vidFeature_path
    self.txt = txt
    self.vocab = vocab
    self.ids = []
    cnt = 0
    with open(self.txt, encoding="utf8") as fp: 
      Lines = fp.readlines() 
      for line in Lines:
        pair = line.split('\t')
        self.ids.append(pair[0])
        cnt += 1


  def __getitem__(self, index):
    caption = ''
    id = self.ids[index]
    #print(id)
    with open(self.txt, encoding="utf8") as fp: 
      Lines = fp.readlines() 
      for line in Lines:
        pair = line.split('\t')
        if pair[0] == id:
          caption = pair[1]
          break
   # print(caption)

    caption = "".join(i for i in caption if i in [".","।"] or 2432 <= ord(i) <= 2559 or ord(i)== 32)
    re.sub(' +', ' ', caption)

    t = Tokenizers()
    tokens = t.bn_word_tokenizer(str(caption))
    #tokens = nltk.tokenize.word_tokenize(str(caption).lower())
    tokens = [word for word in tokens]
    caption = []
    caption.append(vocab('<start>'))
    caption.extend([vocab(token) for token in tokens])
    caption.append(vocab('<end>'))
    target = torch.Tensor(caption)

    
    # getting rid of the caption no. in the id (vidID_start_end_captionNo)
    words = id.split('_')
    vid_with_length = words[0] 
    for i in range(1,len(words)-1):
     vid_with_length = vid_with_length + "_" + words[i] 
    
    
    with h5py.File(self.root, 'r') as hdf:
      temp = hdf.get(vid_with_length)
      feature = torch.tensor(temp)

    with h5py.File(self.vidFeature_path, 'r') as hdf:
      temp = hdf.get(vid_with_length)
      vidFeature = torch.tensor(temp)
      vidFeature = torch.max(vidFeature, 0)[0]                        # max pooling from resnext features
    return feature, vidFeature,  target

  def __len__(self):
   return len(self.ids)


def collate_fn(data):

  data.sort(key=lambda x: len(x[2]), reverse=True)
  features, vidFeatures, captions = zip(*data)

  features = torch.stack(features, 0)
  vidFeatures = torch.stack(vidFeatures, 0)

  lengths = [len(cap) for cap in captions]
  targets = torch.zeros(len(captions), max(lengths)).long()

  for i, cap in enumerate(captions):
      end = lengths[i]
      targets[i, :end] = cap[:end]        
  return features, vidFeatures, targets, lengths


def get_loader(root, resnext_file_path, txt, vocab,batch_size, shuffle, num_workers):
  msvd = msvdDataset(root,resnext_file_path,txt,vocab)

  data_loader = torch.utils.data.DataLoader(dataset=msvd, 
                                  batch_size=batch_size,
                                  shuffle=shuffle,
                                  num_workers=num_workers,
                                  collate_fn=collate_fn)

  return data_loader



# ***Evaluation***

In [12]:
!pip install git+https://github.com/salaniz/pycocoevalcap

  Cloning https://github.com/salaniz/pycocoevalcap to /tmp/pip-req-build-abcohxsm
  Running command git clone -q https://github.com/salaniz/pycocoevalcap /tmp/pip-req-build-abcohxsm
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312215 sha256=fdc9e6ffc1c5e09eb7303e2faa61f4a3bed5018184aff56cf66a83cf60cb78e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-wdax5n56/wheels/2d/53/14/4c0bcdeff69d68affb02b89cbbb7ba681fbbaaa5aaeb1405c0
Successfully built pycocoevalcap


In [13]:
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.spice.spice import Spice
import json

In [14]:
def bleu(ground, hyp):
    scorer = Bleu(n=4)
    score, scores = scorer.compute_score(ground, hyp)
    return score
    #print('bleu = %s' % score)

def cider(ground, hyp):
    scorer = Cider()
    (score, scores) = scorer.compute_score(ground, hyp)
    return score
    #print('cider = %s' % score)

def meteor(ground, hyp):
    scorer = Meteor()
    score, scores = scorer.compute_score(ground, hyp)
    return score
    #print('meteor = %s' % score)

def rouge(ground, hyp):
    scorer = Rouge()
    score, scores = scorer.compute_score(ground, hyp)
    return score
    #print('rouge = %s' % score)


In [15]:
def eval(encoder, decoder, encoder_path, decoder_path,mode = "validation"):

  if mode == "validation":
    txt = '/content/drive/MyDrive/Thesis/code/models/validation.txt'
  else:
    txt = '/content/drive/MyDrive/Thesis/code/models/test.txt'


  encoder.load_state_dict(torch.load(encoder_path))
  decoder.load_state_dict(torch.load(decoder_path))
  
  
  #df = pd.read_csv('/content/drive/My Drive/Thesis/MSR Video Description Corpus.csv')
  df = pd.read_excel(caption_path)

  ref_dict = {}
  hyp_dict = {}
  with open(txt) as fp: 
        Lines = fp.readlines() 
        for cnt,line in enumerate(Lines):
          line = line.strip('\n')
          splited_name = line.split('_')
          start = splited_name[-2]
          end = splited_name[-1]
          extention_len = len(start+end)+2
          only_vid_name = line[:-extention_len]
    
          captions = df[df['VideoID'] == only_vid_name]
          #captions = captions[captions['Language'] == 'English']
          captions = captions[captions['Start'] == int(start)]
          captions = captions['Description']
          captions = captions.to_list()
          if len(captions) ==0 :
            continue

          for i,item in enumerate(captions):
            s = item
            if isinstance(s, float):
              captions.remove(captions[i])
              continue
            s = s.lower()
            s = s.replace('\t','')
            s = s.replace('\n','')
            s = s.translate(str.maketrans('', '', string.punctuation))
            captions[i] = s
            
            

          ref_dict[line] = captions 


          with h5py.File('/content/drive/MyDrive/Thesis/code/features.h5', 'r') as hdf:
            data = hdf.get(line)
            image_tensor = torch.tensor(data)
            image_tensor = image_tensor.unsqueeze(0)

          
          with h5py.File('/content/drive/MyDrive/Thesis/code/resnext_101_features.h5', 'r') as hdf:
            temp = hdf.get(line)
            vidFeature = torch.tensor(temp)
            vidFeature = torch.max(vidFeature, 0)[0].unsqueeze(0)  

          feature = encoder(image_tensor,vidFeature)
          sampled_ids = decoder.sample(feature)
          sampled_ids = sampled_ids[0].numpy()

          sampled_caption = []
          for word_id in sampled_ids:
              word = vocab.idx2word[word_id]
              if word == '<end>':
                  break
              if word == '<start>':
                  continue
              
              sampled_caption.append(word)
              
          sentence = ' '.join(sampled_caption)
          hyp_dict[line] = [sentence]
          print(sentence)


  with open('/content/drive/MyDrive/Thesis/code/modelsground.json', 'w') as fp:
    json.dump(ref_dict, fp)

  with open('/content/drive/MyDrive/Thesis/code/modelshyp.json', 'w') as fp:
    json.dump(hyp_dict, fp)

  with open(model_path+'ground.json', 'w') as fp:
    json.dump(ref_dict, fp)

  with open(model_path+'hyp.json', 'w') as fp:
    json.dump(hyp_dict, fp)

   
  with open('/content/drive/MyDrive/Thesis/code/modelsground.json', 'r') as file:
    ground = json.load(file)
  with open('/content/drive/MyDrive/Thesis/code/modelshyp.json', 'r') as file:
    hyp = json.load(file)

  bleu_score = bleu(ground, hyp)
  #meteor_score = meteor(ground, hyp)
  meteor_score = -1
  cider_score = cider(ground, hyp)
  rouge_score = rouge(ground, hyp)

  return bleu_score , meteor_score, cider_score, rouge_score

# ***Bangla Word2Vec Model-1***




In [16]:
class EncoderBiLSTM(nn.Module):
    def __init__(self, image_feature_dimension, vid_feature_dimension, encoder_hidden_size, embed_size, encoder_dropout):
       
        super(EncoderBiLSTM, self).__init__()
        self.bilstm = nn.LSTM(image_feature_dimension, encoder_hidden_size, num_layers = 1, batch_first=True, bidirectional = True )
        self.linear = nn.Linear(2*encoder_hidden_size + vid_feature_dimension,embed_size)
        self.activation = nn.Tanh()
        self.dropout = nn.Dropout(p=encoder_dropout)
    
    def forward(self, image_features, vid_features):
        output, _ = self.bilstm(image_features)
        output = output[:,-1,:]

        output = torch.cat([output, vid_features], dim=1)
        output = self.activation(output)
        output = self.dropout(output)
        output = self.linear(output.float())
      
        return output


class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers,dropout,max_seq_length=20):
        """Set the hyper-parameters and build the layers."""
        super(DecoderRNN, self).__init__()
        self.embed = nn.Linear(300, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, dropout = dropout)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.max_seg_length = max_seq_length
        
    def forward(self, features, captions, lengths):
        """Decode image feature vectors and generates captions."""

        temp = captions.numpy()

        row,col = temp.shape

        target = []
        for r in range(row):
          single = []
          for c in range(col):
            single.append(embedding[temp[r][c]])
          target.append(single)
        target = np.array(target)
        captions = torch.from_numpy(target)

        embeddings = self.embed(captions)
        embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True) 
        hiddens, _ = self.lstm(packed)
        outputs = self.linear(hiddens[0])
        return outputs
    
    def sample(self, features, states=None):
        """Generate captions for given image features using greedy search."""
        sampled_ids = []
        inputs = features.unsqueeze(1)
        for i in range(self.max_seg_length):
            hiddens, states = self.lstm(inputs, states)          # hiddens: (batch_size, 1, hidden_size)
            outputs = self.linear(hiddens.squeeze(1))            # outputs:  (batch_size, vocab_size)
            _, predicted = outputs.max(1)                        # predicted: (batch_size)
            sampled_ids.append(predicted)
            predicted = predicted.numpy()

            tar = []
            for item in predicted:
              tar.append(embedding[item])
            tar = np.array(tar)
            predicted = torch.from_numpy(tar)
            
            inputs = self.embed(predicted)                       # inputs: (batch_size, embed_size)
            inputs = inputs.unsqueeze(1)                         # inputs: (batch_size, 1, embed_size)
        sampled_ids = torch.stack(sampled_ids, 1)                # sampled_ids: (batch_size, max_seq_length)
        return sampled_ids

Plot Graph

In [17]:
def plot(X,Y,label):
  plt.plot(X, Y, label=label)
  plt.xlabel("Epoch")
  plt.ylabel("Score")
  # Add a legend
  plt.legend(loc='upper left')
  
  plt.savefig(model_path+label+'.png')
  # Show the plot
  plt.show()

Copying Files

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
!cp '/content/drive/MyDrive/Thesis/code/models/training.txt' 'training.txt'
!cp '/content/drive/MyDrive/Thesis/code/models/validation.txt' 'validation.txt'
!cp '/content/drive/MyDrive/Thesis/code/models/test.txt' 'test.txt'
!cp '/content/drive/MyDrive/Thesis/code/features.h5' 'features.h5'
!cp '/content/drive/MyDrive/Thesis/code/resnext_101_features.h5' 'resnext_101_features.h5'
!cp '/content/drive/MyDrive/Thesis/code/bangla_w2v_embeddings.pkl' 'bangla_embedding.pkl'

# **Training**

In [21]:
image_feature_dimension = 4096
vid_feature_dimension = 2048
encoder_hidden_size = 1000
encoder_dropout = 0.1
decoder_hidden_size = 1000
embed_size = 1000
learning_rate = 0.0002
decoder_num_layers = 2
num_epochs = 1
decoder_dropout = 0.5
batch_size = 500


with open(vocab_path, 'rb') as f:
        vocab = pickle.load(f)

with open(embedding_path, 'rb') as f:
        embedding = pickle.load(f)

encoder = EncoderBiLSTM(image_feature_dimension, vid_feature_dimension, encoder_hidden_size, embed_size, encoder_dropout)
decoder = DecoderRNN(embed_size, decoder_hidden_size, len(vocab), decoder_num_layers, decoder_dropout)
    
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
params = list(decoder.parameters()) + list(encoder.parameters()) 
optimizer = torch.optim.Adam(params, lr= learning_rate)


data_loader = get_loader(h5_file_path, resnext_file_path, training_file_path, vocab, 
                             batch_size,
                             shuffle=True, num_workers=2) 


In [22]:
if path.exists(model_path+'model_stat.json'):

  with open(model_path+'model_stat.json', 'r') as file:
    model_stat = json.load(file)

  starting_epoch = model_stat["epoch"]
  epoch_list = model_stat["epoch_list"]
  bleu3_list = model_stat["bleu3_list"]
  bleu4_list = model_stat["bleu4_list"]
  meteor_list = model_stat["meteor_list"]
  cider_list = model_stat["cider_list"]
  rouge_list = model_stat["rouge_list"]

  
  encoder_path = checkpoint_path+'encoder-'+str(starting_epoch)+'.ckpt'
  decoder_path = checkpoint_path+'decoder-'+str(starting_epoch)+'.ckpt'
  optimizer_path = checkpoint_path+'optimizer-'+str(starting_epoch)+'.ckpt'

  encoder.load_state_dict(torch.load(encoder_path))
  decoder.load_state_dict(torch.load(decoder_path))
  optimizer.load_state_dict(torch.load(optimizer_path))

else:
  starting_epoch = 0
  epoch_list = []
  bleu3_list = []
  bleu4_list = []
  meteor_list = []
  cider_list = []
  rouge_list = []
  model_stat = {}



In [23]:
# Train the models
total_step = len(data_loader)

for epoch in range(starting_epoch,num_epochs):
  now = time.time()
  for i,(img_features, vid_features, captions, lengths) in enumerate(data_loader):
    
    targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]
              
          
    features = encoder(img_features, vid_features)
    outputs = decoder(features, captions, lengths)
    loss = criterion(outputs, targets)
    decoder.zero_grad()
    encoder.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%10 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Perplexity: {:5.4f}'
                          .format(epoch+1, num_epochs, i+1, total_step, loss.item(), np.exp(loss.item())))
    
  later = time.time()
  difference = int(later - now)
  print('time required for epoch {} : {}'.format(epoch+1, difference))

  torch.save(decoder.state_dict(), os.path.join(
                    checkpoint_path, 'decoder-{}.ckpt'.format(epoch+1)))
  torch.save(encoder.state_dict(), os.path.join(
                    checkpoint_path, 'encoder-{}.ckpt'.format(epoch+1)))
  torch.save(optimizer.state_dict(), os.path.join(
                    checkpoint_path, 'optimizer-{}.ckpt'.format(epoch+1)))

  encoder_path = checkpoint_path+'encoder-'+str(epoch+1)+'.ckpt'
  decoder_path = checkpoint_path+'decoder-'+str(epoch+1)+'.ckpt'
  optimizer_path = checkpoint_path+'optimizer-'+str(epoch+1)+'.ckpt'

  bleu_score , meteor_score, cider_score, rouge_score = eval(encoder, decoder, encoder_path, decoder_path)

  print('Score : bleu3 : {}, bleu4 : {}, meteor : {}, cider : {}, rouge : {}'.format(bleu_score[2] ,bleu_score[3], meteor_score, cider_score, rouge_score))


  epoch_list.append(epoch+1)
  bleu3_list.append(bleu_score[2])
  bleu4_list.append(bleu_score[3])
  meteor_list.append(meteor_score)
  rouge_list.append(rouge_score)
  cider_list.append(cider_score)

  
  model_stat["epoch"] = epoch + 1
  model_stat["epoch_list"] = epoch_list
  model_stat["bleu3_list"] = bleu3_list
  model_stat["bleu4_list"] = bleu4_list
  model_stat["meteor_list"] = meteor_list
  model_stat["rouge_list"] = rouge_list
  model_stat["cider_list"] = cider_list


  with open(model_path+'/model_stat.json', 'w') as fp:
    json.dump(model_stat, fp)

  plot(epoch_list,bleu3_list,'Bangla Model-1 Bleu3')
  plot(epoch_list,bleu4_list,'Bangla Model-1 Bleu4')
  plot(epoch_list,meteor_list,'Bangla Model-1 Meteor')
  plot(epoch_list,cider_list,'Bangla Model-1 CIDEr')
  plot(epoch_list,rouge_list,'Bangla Model-1 rouge')


  

Epoch [1/1], Step [1/99], Loss: 6.6328, Perplexity: 759.5862
Epoch [1/1], Step [11/99], Loss: 2.0144, Perplexity: 7.4960
Epoch [1/1], Step [21/99], Loss: 1.1241, Perplexity: 3.0774
Epoch [1/1], Step [31/99], Loss: 0.8254, Perplexity: 2.2829
Epoch [1/1], Step [41/99], Loss: 0.6878, Perplexity: 1.9893
Epoch [1/1], Step [51/99], Loss: 0.6199, Perplexity: 1.8588
Epoch [1/1], Step [61/99], Loss: 0.6462, Perplexity: 1.9082
Epoch [1/1], Step [71/99], Loss: 0.6071, Perplexity: 1.8351


KeyboardInterrupt: ignored

In [26]:
  torch.save(decoder.state_dict(), os.path.join(
                    checkpoint_path, 'decoder-{}.ckpt'.format(epoch+1)))
  torch.save(encoder.state_dict(), os.path.join(
                    checkpoint_path, 'encoder-{}.ckpt'.format(epoch+1)))
  torch.save(optimizer.state_dict(), os.path.join(
                    checkpoint_path, 'optimizer-{}.ckpt'.format(epoch+1)))

  encoder_path = checkpoint_path+'encoder-'+str(epoch+1)+'.ckpt'
  decoder_path = checkpoint_path+'decoder-'+str(epoch+1)+'.ckpt'
  optimizer_path = checkpoint_path+'optimizer-'+str(epoch+1)+'.ckpt'

  bleu_score , meteor_score, cider_score, rouge_score = eval(encoder, decoder, encoder_path, decoder_path)

  print('Score : bleu3 : {}, bleu4 : {}, meteor : {}, cider : {}, rouge : {}'.format(bleu_score[2] ,bleu_score[3], meteor_score, cider_score, rouge_score))

<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk

In [ ]:
with open('ground.json', 'r') as file:
    ground = json.load(file)
with open('hyp.json', 'r') as file:
    hyp = json.load(file)


#bleu(ground, hyp)
meteor(ground, hyp)
#cider(ground, hyp)
#rouge(ground, hyp)

0.2222287086139124

In [ ]:
test_epoch = 1
test_encoder_path = checkpoint_path+'encoder-'+str(test_epoch)+'.ckpt'
test_decoder_path = checkpoint_path+'decoder-'+str(test_epoch)+'.ckpt'

eval(encoder, decoder, test_encoder_path, test_decoder_path)


একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন লোক একটি
একজন ল

([0.8064465921283501,
  0.5597153695776297,
  0.33313926506801333,
  0.013808922974469465],
 -1,
 0.06028205175137769,
 0.46364259871367325)